In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import matplotlib.pyplot as plt
%matplotlib inline
from multiprocess import Pool
import numpy as np

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *

Setting environment for Precision WorkStation


No vtk


In [2]:
atlas_spec = dict(name='atlasV6',
                   vol_type='score'    ,               
                    resolution='10.0um'
                   )

In [3]:
atlas_structures_wrt_canonicalAtlasSpace_atlasResol = \
DataManager.load_original_volume_all_known_structures_v3(atlas_spec, in_bbox_wrt='canonicalAtlasSpace',
                                                        out_bbox_wrt='canonicalAtlasSpace')

Prior structure/index map not found. Generating a new one.


In [4]:
atlas_structure_centroids_wrt_canonicalAtlasSpace_atlasResol = get_structure_centroids(vol_origin_dict=atlas_structures_wrt_canonicalAtlasSpace_atlasResol)

In [5]:
atlas_anchor1_wrt_canonicalAtlasSpace_atlasResol = \
np.r_[atlas_structure_centroids_wrt_canonicalAtlasSpace_atlasResol['12N'][:2], 0]
atlas_anchor2_wrt_canonicalAtlasSpace_atlasResol = \
np.r_[atlas_structure_centroids_wrt_canonicalAtlasSpace_atlasResol['3N_L'][:2], 0]

In [7]:
stack = 'CHATM2'

In [8]:
if stack == 'CHATM2':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([565,246,179]) * 20. / 10. # 12N
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([388,183,179]) * 20. / 10. # 3N_L
elif stack == 'CHATM3':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([740,258,230]) * 20. / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([550,195,230]) * 20. / 10.

In [16]:
from registration_utilities import *

In [17]:
def align_anchors(pm1, pm2, pf1, pf2):
    
    t = ((pf1 + pf2) / 2. - (pm1 + pm2) / 2.)

    subject_d = pf1 - pf2
    atlas_d = pm1 - pm2

    subject_d_n = subject_d / np.linalg.norm(subject_d)
    atlas_d_n = atlas_d / np.linalg.norm(atlas_d)

    R = R_align_two_vectors(atlas_d_n, subject_d_n)

    T = np.zeros((3,4))
    T[:3, :3] = R
    T[:3, 3] = t
    
    return T

In [29]:
T_atlas_subject_atlasResol = align_anchors(atlas_anchor1_wrt_canonicalAtlasSpace_atlasResol, atlas_anchor2_wrt_canonicalAtlasSpace_atlasResol,
             subject_anchor1_wrt_wholebrainWithMargin_atlasResol, subject_anchor2_wrt_wholebrainWithMargin_atlasResol)

In [31]:
print T_atlas_subject_atlasResol

[[ 9.90767489e-01  1.35582009e-01  0.00000000e+00  9.22098379e+02]
 [-1.35582009e-01  9.90767489e-01  0.00000000e+00  4.48968802e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  3.58000000e+02]]


In [20]:
atlas_structures_wrt_wholebrainWithMargin_atlasResol = \
{name_s: transform_volume_v4(volume=vo, transform=T_atlas_subject_atlasResol, return_origin_instead_of_bbox=True)
for name_s, vo in atlas_structures_wrt_canonicalAtlasSpace_atlasResol.iteritems()}

/home/yuncong/Brain/utilities/registration_utilities.py:1545: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(volume_m_aligned_to_f.dtype, np.float):
Interpolating/filling sparse volume: 0.17 seconds.
Interpolating/filling sparse volume: 0.62 seconds.
Interpolating/filling sparse volume: 0.16 seconds.
Interpolating/filling sparse volume: 0.11 seconds.
Interpolating/filling sparse volume: 0.03 seconds.
Interpolating/filling sparse volume: 0.36 seconds.
Interpolating/filling sparse volume: 0.23 seconds.
Interpolating/filling sparse volume: 0.09 seconds.
Interpolating/filling sparse volume: 0.03 seconds.
Interpolating/filling sparse volume: 0.39 seconds.
Interpolating/filling sparse volume: 0.54 seconds.
Interpolating/filling sparse volume: 0.14 seconds.
Interpolating/filling sparse volume: 0.11 seconds.
Interpolating/filling sparse volume: 

In [21]:
atlas_structures_bbox_wrt_wholebrainWithMargin_atlasResol = \
{name_s: (o[0], o[0] + v.shape[1] - 1, o[1], o[1] + v.shape[0] - 1, o[2], o[2] + v.shape[2] - 1)
 for name_s, (v, o) in atlas_structures_wrt_wholebrainWithMargin_atlasResol.iteritems()}

In [22]:
# int(np.round(metadata_cache['section_limits'][stack].mean()))

In [23]:
converter = CoordinatesConverter(stack=stack)

In [24]:
intensity_volume_spec = dict(name=stack, resolution='10.0um', prep_id='wholebrainWithMargin', vol_type='intensity')
_, thumbnail_volume_origin_wrt_wholebrain_dataResol = DataManager.load_original_volume_v2(intensity_volume_spec, return_origin_instead_of_bbox=True)

In [25]:
converter.derive_three_view_frames(base_frame_name='wholebrainWithMargin', 
                                   origin_wrt_wholebrain_um=thumbnail_volume_origin_wrt_wholebrain_dataResol)

In [26]:
converter.register_new_resolution('image', convert_resolution_string_to_um(resolution='raw', stack=stack))

In [27]:
atlas_structures_wrt_wholebrainWithMargin_section_limits = {}

for name_s, b in atlas_structures_bbox_wrt_wholebrainWithMargin_atlasResol.iteritems():
#     print name_s
    atlas_structures_wrt_wholebrainWithMargin_section_limits[name_s] = \
    converter.convert_frame_and_resolution(p=[[np.nan, np.nan, b[4]], [np.nan, np.nan, b[5]]], 
                                       in_wrt=('wholebrainWithMargin', 'sagittal'),
                                      in_resolution='10.0um',
                                      out_wrt='wholebrain',
                                      out_resolution='image_image_section')[:,2].astype(np.int)

In [28]:
save_json(atlas_structures_wrt_wholebrainWithMargin_section_limits, '/home/yuncong/' + stack + '_structure_section_limits.json')

In [ ]:
stack_f_spec = dict(name='CHATM3',
                   vol_type='score',
                   detector_id=799,
                   structure=convert_to_original_name(structure_m),
                    resolution='10.0um'
                   )

local_alignment_spec = dict(stack_m=stack_m_spec, 
                      stack_f=stack_f_spec,
                      warp_setting=27)